In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet1.zip', 'r') as zip_ref:
        zip_ref.extractall('./DataSet1')
except:
    print("Using Local Machine")
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Mounted at /content/drive
Cloning into 'yolov5'...
remote: Enumerating objects: 15598, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (149/149), done.


In [ ]:
# Include all packages
import gc
import cv2
import shutil
import random
import numpy as np
import pandas as pd
from time import time
from copy import deepcopy

from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from yolov5.models.yolo import Model
from sklearn.model_selection import train_test_split

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
def CannyEdge(capturedImage):
    grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
    constrastKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5) )
    topHat = cv2.morphologyEx(grayScale, cv2.MORPH_TOPHAT, constrastKernel)
    blackHat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, constrastKernel)
    grayScale = grayScale + topHat - blackHat
    bilateralFilter = cv2.bilateralFilter(grayScale, 11, 17, 17)
    imageMedian = np.median(capturedImage)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))
    cannyEdgeImage = cv2.Canny(bilateralFilter, lowerThreshold, upperThreshold)
    cannyEdgeImage = cv2.bitwise_not(cannyEdgeImage)
    cannyEdgeImage = cv2.cvtColor(cannyEdgeImage, cv2.COLOR_GRAY2BGR)
    return cannyEdgeImage

In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [uniqueSigns.index(row[5]), row[1], row[2], row[3], row[4]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, numClasses: int, resize: tuple) -> tuple:
    resizedImages = [[] for i in range(numClasses)]
    newAnnotations = [[] for i in range(numClasses)]
    for i, image in enumerate(images):
        [classIndex, x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImages[classIndex].append(resizedImage)
        newAnnotations[classIndex].append(
            [x1New, y1New, x2New, y2New])
    
    return resizedImages, newAnnotations


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:

def TargetstoTensors(targets, batchSize, numAnchors, gridSizes):
    targetObj = []
    targetBox = []
    for grid_size in gridSizes:
        targetObj.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 1)))
        targetBox.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 4)))

    for batch_index, target in enumerate(targets):
        x1, y1, x2, y2 = target.long()
        x_center, y_center, width, height = (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1

        for i, grid_size in enumerate(gridSizes):
            x_cell, y_cell = int(x_center * grid_size), int(y_center * grid_size)
            anchor = 0
            try:
                targetObj[i][batch_index, anchor, y_cell, x_cell, 0] = 1
                targetBox[i][batch_index, anchor, y_cell, x_cell] = torch.tensor([x_center, y_center, width, height])
            except Exception as e:
                pass
    return targetObj, targetBox

In [ ]:

class SignboardLoss(nn.Module):
    def __init__(self, num_anchors=3):
        super(SignboardLoss, self).__init__()
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        objectLoss = torch.tensor(0.0, device=preds[0].device)
        boxLoss = torch.tensor(0.0, device=preds[0].device)
        batchSize = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetObjList, targetBoxList = TargetstoTensors(targets, batchSize, self.num_anchors, gridSizes)

        for i, pred in enumerate(preds):
            targetObj = targetObjList[i].to(pred.device)
            targetBox = targetBoxList[i].to(pred.device)

            objectLoss += nn.BCEWithLogitsLoss()(pred[..., 4:5], targetObj)
            boxLoss += nn.MSELoss()(pred[..., :4], targetBox)

        total_loss = objectLoss + boxLoss
        return total_loss

In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    ckpt = torch.load(f'yolov5{version}.pt', map_location=device)
    ckpt_model_dict = ckpt['model'].state_dict()
    compatible_weights = {k: v for k, v in ckpt_model_dict.items() if k in model.state_dict() and model.state_dict()[k].shape == v.shape}
    model.load_state_dict(compatible_weights, strict=False)
    model.hyp = ckpt['model'].hyp
    return model



In [ ]:
def TrainEpoch(model, dataLoader, optimizer, lossFunction, device):
    # print("Training Epoch")
    model.train()
    runningLoss = 0
    dataLoaderLen = len(dataLoader)
    for i, (inputs, targets) in enumerate(dataLoader):
        # inputs = inputs.permute(2, 0, 1)
        inputs = inputs.permute(0, 3, 1, 2)
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = lossFunction(outputs, targets)
            loss.backward()
            optimizer.step()

        runningLoss += loss.item() * inputs.size(0)
        # if(((i*100)//dataLoaderLen) % 10 == 0):
        #     print((i*100//dataLoaderLen), end="%,")
    epochLoss = runningLoss / dataLoaderLen
    return model, epochLoss


In [ ]:
def DetectImage(model, inputs, device, conf_thres=0.2, iou_thres=0.5):
    model.eval()
    inputs = torch.tensor(inputs, dtype=torch.float32)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.to(device)
    conf_thres = torch.tensor(conf_thres)
    with torch.no_grad():
        output = model(inputs)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        # output = torchvision.ops.nms(output, conf_thres, iou_thres)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        output = output[0]
        box_coordinates = output[..., :4].view(-1, 4)
        confidence_scores = output[..., 4].view(-1)
        nms_indices = torchvision.ops.nms(box_coordinates, confidence_scores, iou_thres)
        output = output.view(-1, output.shape[-1])[nms_indices]
    # Remove the batch dimension
    output = output.squeeze(0)
    return output


In [ ]:

def EvaluateModel( yolov5Model,X_val: list, y_val: list):
    randInt = random.randint(0,len(X_val))
    image = X_val[randInt]
    print(len(X_val))
    image1 = deepcopy(image)
    predictions = DetectImage(yolov5Model, image, device)
    [a1,b1,a2,b2] = y_val[randInt]
    bBoxs = [[a1,b1,a2,b2]]
    machingbBoxes = []
    for pred in predictions:
        x1, y1, x2, y2, m1,m2 = pred[:6]
        m1,m2, x1, y1, x2, y2= int(m1), int(m2),int(x1), int(y1), int(x2), int(y2)
    if(a1 == x1 or a2 == x2 or b1 == y1 or b2 == y2 ):
        if(((x1-x2) >= 17 and (x1-x2) <= 32) and ((y1-y2) >= 31 and (y1-y2)<= 56) ):
            machingbBoxes.append([x1, y1, x2,y2])
      
    if(((x1-x2) >= 17 and (x1-x2) <= 32) and ((y1-y2) >= 31 and (y1-y2)<= 56) ):
        bBoxs.append([x1, y1, x2, y2])
    print("No. Objects detected:" ,len(bBoxs) )
    print("No. Matching Objects detected:" ,len(machingbBoxes) )
    cv2.rectangle(image, (a1, b1), (a2, b2), (0,255,0), 2)
    for bBox in bBoxs[1:]:
        # print(bBox)
        [x1, y1, x2, y2] = bBox
        cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)
    for bBox in machingbBoxes:
        [x1, y1, x2, y2] = bBox
        cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(image1)
    except:
        print("using Local")
        cv2.imshow("Input Image", image1)



In [ ]:
def TrainModel(model, trainDataLoader, valDataLoader, epochs, optimizer, scheduler, lossFunction, device):
    startTime = time()
    for epoch in range(epochs):
        # print("Epoch {}/{}:".format(epoch+1, epochs))
        model, trainingEpochLoss = TrainEpoch(model, trainDataLoader, optimizer, lossFunction, device)
        scheduler.step(trainingEpochLoss)
        if(((epoch*100)/epochs) % 10 == 0):
            endTime = time()
            timeTaken = endTime - startTime
            print((epoch*100//epochs), end="%,")
            print("Training Loss: {:.4f}".format(trainingEpochLoss), end=",")
            print("Time taken: {}min, {}, secs".format(timeTaken//60, int(timeTaken % 60)))
            startTime = time()
    
    print()
    
    
    print("Training complete.")
    return model


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 300
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Using {} device".format(device))

In [ ]:
print("Downloading Weights of yolo5 Verion ", yolo5Version)
weightsURL = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(yolo5Version)
!wget {weightsURL}

In [ ]:
images, annotations, numClasses = LoadDataSet("./DataSet1")
print(numClasses)

In [ ]:
# X_train, X_val, y_train, y_val = PreProcessDataSet(
#     images, annotations, batchSize, inputShape)
X_train, y_train = PreProcessDataSet(
    images, annotations, numClasses, inputShape)
X_val = [[] for i in range(numClasses)]
y_val = [[] for i in range(numClasses)]
del images
del annotations
gc.collect()

In [ ]:
# try:
#     from google.colab.patches import cv2_imshow
#     cv2_imshow(X_train[10])
# except:
#     print("using Local")
#     # cv2.imshow("Input Image", image)

In [ ]:
yolov5Model = CreateYolov5Model(1,yolo5Version)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.01)
yolov5LossFunction= SignboardLoss()
yolov5Model = yolov5Model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)


In [ ]:
for i in range(numClasses):
    print( "training class", i, "with number of samples:", len(X_train[i]))
    trainDataLoader, valDataLoader = CreateDataLoaders(
        X_train[i], X_val[i], y_train[i], y_val[i], batchSize)
    epochs = (900//len(X_train[i]))*100
    print("epochs", epochs)
    yolov5Model = TrainModel(yolov5Model, trainDataLoader,valDataLoader, epochs, optimizer, scheduler, yolov5LossFunction, device)
    EvaluateModel( yolov5Model,X_train[i], y_train[i])
    date = datetime.now()
    date = date.strftime("%m-%d-%H")
    torch.save(yolov5Model.state_dict(), 'yolov5SingleModelv2' + date +'-class-'+str(i)+'.pth')
    shutil.copy('/content/yolov5SingleModelv2' + date +'-class-'+str(i)+'.pth', '/content/drive/MyDrive/DL Project/Trained Models/yolov5SingleModelv2' + date +'-class-'+str(i)+'.pth')


In [ ]:
date = datetime.now()
date = date.strftime("%m-%d-%H")
torch.save(yolov5Model.state_dict(), 'yolov5SingleModelv2' + date +'.pth')
shutil.copy('/content/yolov5SingleModelv2' + date +'.pth', '/content/drive/MyDrive/DL Project/Trained Models/yolov5SingleModelv2' + date +'.pth')
